# Why yolort?

- TorchVision 风格代码，容易理解，方便魔改。
- 无缝 load YOLOv5 官方训练好的 checkpoints, 保证跟 YOLOv5 结构一致.
- 充分的单元测试.
- 部署友好, TensorRT / ONNX Runtime / LibTorch / ncnn 多后端支持。

## 安装 yolort

In [1]:
%pip install -q yolort

Note: you may need to restart the kernel to use updated packages.


In [2]:
from typing import List

import torch
from torch import nn, Tensor

## 部分代码细节
1. [LetterBox](https://github.com/zhiqwang/yolov5-rt-stack/blob/7cdef76166b2497abf626d12ca74f2c4401f80e2/yolort/models/transform.py#L112) 使用 PyTorch 原生算子实现，通过 `torch.nn.functional.interpolate` 和 `pad` 算子实现。方便导出到 TorchScript 及 ONNX 等格式。
1. 兼容 P5/P6 的 [PathAggregationNetwork](https://github.com/zhiqwang/yolov5-rt-stack/blob/7cdef76166b2497abf626d12ca74f2c4401f80e2/yolort/models/path_aggregation_network.py#L44) 实现，充分体现出 PAN 结构中的对称性。
1. 极简实现的后处理 [PostProcess](https://github.com/zhiqwang/yolov5-rt-stack/blob/7cdef76166b2497abf626d12ca74f2c4401f80e2/yolort/models/box_head.py#L360).

### 兼容 YOLOv5 原版网络结构

In [3]:
from yolort.v5 import attempt_download
from yolort.models._checkpoint import load_from_ultralytics

# yolov5s6.pt is downloaded from 'https://github.com/ultralytics/yolov5/releases/download/v6.1/yolov5s6.pt'
model_path = "yolov5s6.pt"

checkpoint_path = attempt_download(model_path)
version = "r6.0"

model_info = load_from_ultralytics(checkpoint_path, version=version)

### 获取 Backbone

In [4]:
from yolort.models.backbone_utils import darknet_pan_backbone

backbone_name = f"darknet_{model_info['size']}_{version.replace('.', '_')}"
depth_multiple = model_info["depth_multiple"]
width_multiple = model_info["width_multiple"]
use_p6 = model_info["use_p6"]
backbone = darknet_pan_backbone(backbone_name, depth_multiple, width_multiple, version=version, use_p6=use_p6)

### 获取完整的 YOLOv5 网络结构

In [5]:
from yolort.models.box_head import YOLOHead

class YOLO(nn.Module):
    def __init__(self, backbone: nn.Module, strides, num_anchors, num_classes: int):
        super().__init__()
        self.backbone = backbone
        self.head = YOLOHead(backbone.out_channels, num_anchors, strides, num_classes)

    def forward(self, samples: Tensor) -> List[Tensor]:

        # get the features from the backbone
        features = self.backbone(samples)

        # compute the yolo heads outputs using the features
        head_outputs = self.head(features)
        return head_outputs

In [6]:
strides = model_info["strides"]
num_anchors = len(model_info["anchor_grids"][0]) // 2 
num_classes = model_info["num_classes"]
model = YOLO(backbone, strides, num_anchors, num_classes)

model.load_state_dict(model_info["state_dict"])

model = model.eval()

In [7]:
inputs = torch.rand(1, 3, 640, 640)

outs = model(inputs)

### 完整的部署支持方案

In [8]:
from torch.fx import symbolic_trace

In [9]:
model_traced = symbolic_trace(model)